In [1]:
sample = 'dabAcCaCBAcCcaDA'

In [2]:
polymer = sample[:]
output = ''
i = 0
while i < len(polymer)-1:
    u, v = polymer[i], polymer[i + 1]

    if u.lower() == v.lower() and u != v:
        # reaction
        left = polymer[:i]
        right = polymer[i+2:]
        polymer = left + right
        # move on
        i = len(left) - 1
        
    else:
        # no reaction
        i += 1
    #print(i, u, v, polymer)
len(polymer)

10

In [3]:
polymer == 'dabCBAcaDA'

True

Let's now try with the real input.

In [4]:
real_polymer = [line.strip() for line in open('input05').readlines()][0]

In [5]:
real_polymer[:10]
real_polymer[-10:]

'tTRroUkKuI'

In [6]:
def react_until_end_fast(polymer):
    i = 0
    while i < len(polymer)-1:
        u, v = polymer[i], polymer[i + 1]

        if u.lower() == v.lower() and u != v:
            # reaction
            left = polymer[:i]
            right = polymer[i+2:]
            assert(len(left + right) < len(polymer))
            polymer = left + right

            # move on
            i = len(left) - 1
            if i < 0:
                i = 0
        else:
            # no reaction
            i += 1
        #print(i, u, v, polymer)
    len(polymer)
    return polymer

In [7]:
len(react_until_end_fast(real_polymer))

9686

# Plan B 

It seems that I'm stuck here. Let's try plan B.

We will write a function that scans the chain from its start and creates the next chain by reacting the first element it finds.

In [8]:
def react_once(polymer):
    i = 0
    while i < len(polymer)-1:
        u, v = polymer[i], polymer[i + 1]

        if u.lower() == v.lower() and u != v:
            # reaction
            left = polymer[:i]
            right = polymer[i+2:]
            assert(len(left + right) < len(polymer))
            polymer = left + right
            return polymer
        else:
            # no reaction
            i += 1
    return polymer

In [9]:
react_once('aA')

''

In [10]:
react_once('abBA')

'aA'

In [11]:
react_once('abAB')

'abAB'

In [12]:
react_once('aabAAB')

'aabAAB'

In [13]:
react_once('dabAcCaCBAcCcaDA')

'dabAaCBAcCcaDA'

Now let's write a function that reacts until the reaction can not take place anymore.

In [14]:
def react_until_end(polymer):
    """Super slow!"""
    n = len(polymer)
    polymer = react_once(polymer)
    while len(polymer) < n:
        n = len(polymer)
        polymer = react_once(polymer)
    return polymer

In [15]:
react_until_end('dabAcCaCBAcCcaDA')

'dabCBAcaDA'

In [16]:
len('dabCBAcaDA')

10

In [17]:
len(react_until_end(real_polymer))

9686

# Part B 

Let's write a function that replaces a unit.

In [18]:
def remove_unit(polymer, unit_type):
    return polymer.replace(unit_type.lower(), '').replace(unit_type.upper(), '')

In [19]:
remove_unit('dabAcCaCBAcCcaDA', 'a')

'dbcCCBcCcD'

In [20]:
for unit_type in 'abcd':
    polymer = remove_unit('dabAcCaCBAcCcaDA', unit_type)
    reacted = react_until_end_fast(polymer)
    print(unit_type, len(reacted))

a 6
b 8
c 4
d 6


In [21]:
lens = []
for unit_type in list(set(real_polymer.lower())):
    polymer = remove_unit(real_polymer, unit_type)
    reacted = react_until_end_fast(polymer)
    lens.append(len(reacted))
    print(unit_type, len(reacted))
min(lens)

z 9268
r 9326
d 9282
g 9314
w 9364
i 9276
l 9314
c 9262
b 9368
f 9316
q 9306
u 9280
j 9336
o 9320
m 9228
h 9316
t 9250
e 9304
s 9284
n 9274
v 9260
p 9322
y 9278
x 9368
a 9272
k 5524


5524